# Notes

This notebook creates subhalo ID and particle ID lists for snapshots for IllustrisTNG simulations.

These can be loaded and used for cross-matching for impact factors to galaxies.

The code is adapted from: 

raven:/u/cwalker/Illustris_FRB_Project/Create_Particle-Subhalo_Matchlists.ipynb

It builds on previous work by creating lists for any desired simulation.

# Imports

In [1]:
import os
import yt
import h5py
import random

import numpy as np
import illustris_python as il

from matplotlib import pyplot as plt
from charlie_TNG_tools import pID2shID


/raven/u/cwalker/virtual_environments/yt-git/yt/utilities/logger.py:4: VisibleDeprecationWarning: The configuration file /u/cwalker/.config/yt/ytrc is deprecated in favor of /u/cwalker/.config/yt/yt.toml. Currently, both are present. Please manually remove the deprecated one to silence this warning.
Deprecated since v4.0.0 . This feature will be removed in v4.1.0
  from yt.config import ytcfg


# Initialise

In [2]:
#simulation to create lists for
sim_to_use = 'TNG100-2'

#snapshots to create lists for
snap_list = [99]

#base path to data
basePath = '/virgo/simulations/IllustrisTNG/{0}/output/'.format(sim_to_use)

#path to simulation hdf5 file
simPath = '/virgo/simulations/IllustrisTNG/{0}/simulation.hdf5'.format(sim_to_use)

#check these exist
print('Testing whether basePath and simPath exist...')
print('basePath exists = {0}'.format(os.path.exists(basePath)))
print('simPath exists = {0}'.format(os.path.exists(simPath)))


Testing whether basePath and simPath exist...
basePath exists = True
simPath exists = True


# Create directories to store matchlists

In [3]:
#check to see if a top directory exists
topdir_name = '/u/cwalker/Illustris_Zhang_Method/Sim_Matchlists/'
topdir_check = os.path.exists(topdir_name)

#if it exists, print
if topdir_check == True:
    print('Top directory ({0}) already exists.'.format(topdir_name))
    
#else, create it.
elif topdir_check == False:
    print('Creating top directory for matchlists at {0}...'.format(topdir_name))
    os.mkdir(topdir_name)
    print('{0} created.'.format(topdir_name))
    
    
#check to see if subdirectory for particular simulation matchlist exists
subdir_name = topdir_name+'Matchlist_dir_{0}'.format(sim_to_use)
subdir_check = os.path.exists(subdir_name)

#if it exists, print
if subdir_check == True:
    print('Directory to hold {0} matchlist ({1} exists.)'.format(sim_to_use,subdir_name))

#else, create it
elif subdir_check == False:
    print('Creating subdirectory {0}...'.format(subdir_name))
    os.mkdir(subdir_name)
    print('{0} created.'.format(subdir_name))

Top directory (/u/cwalker/Illustris_Zhang_Method/Sim_Matchlists/) already exists.
Directory to hold TNG100-2 matchlist (/u/cwalker/Illustris_Zhang_Method/Sim_Matchlists/Matchlist_dir_TNG100-2 exists.)


# Create desired matchlists

In [4]:
#loop over snapshots
for snapshot_number in snap_list:
    print('Creating for {0} snap {1}'.format(sim_to_use,snapshot_number))

     #########################
     #SNAPSHOT DATA LOCATIONS#
     #########################

    offsetFile = '/virgo/simulations/IllustrisTNG/{0}/postprocessing/offsets/offsets_0{1}.hdf5'.format(sim_to_use,snapshot_number)
    data_loc = '/virgo/simulations/IllustrisTNG/{0}/output/snapdir_0{1}/snap_0{1}.0.hdf5'.format(sim_to_use,snapshot_number)
    partIDs_loc = '/Snapshots/{0}/PartType0/ParticleIDs'.format(snapshot_number)
    
    print('Processing snapshot {0} at:\n{1}\n with offset file:\n{2}\n and particle IDs file loc:\n{3}'.format(snapshot_number,data_loc,offsetFile,partIDs_loc))

    #########################
    #CREATE PARTICLE ID LIST#
    #########################
    
    #get all gas particle IDs in snapshot
    with h5py.File(simPath) as f:
        allparts = f[partIDs_loc][:]
    
    #create a list version of every particle ID
    AllPartList = allparts.tolist()
    
    ########################
    #CREATE SUBHALO ID LIST#
    ########################
    
    #load and index cell data
    ds=yt.load(data_loc)
    ds.index
    
    #choose what to ID
    to_ID = np.arange(ds.particle_type_counts['PartType0'])
    partType = 0 #gas
    
    #choose subhalo fields to load
    subhaloFields = ['SubhaloFlag',
                     'SubhaloPos',
                     'SubhaloHalfmassRad',
                     'SubhaloHalfmassRadType',
                     'SubhaloLenType']
    
    #load subhalo catalog
    subhalos=il.groupcat.loadSubhalos(basePath,snapshot_number,fields=subhaloFields)  
    subhalos.keys()
    
    #get subhalo offset file for matching particle and subhalo IDs
    with h5py.File(offsetFile,'r') as f:
        SnapOffsetsSubhalo= np.copy(f['/Subhalo/SnapByType'])
    
    #get subhalo lengths for all gas particles
    SubhaloLenType = np.copy(subhalos['SubhaloLenType'])
    
    #create array of subhaloIDs for every gas particle
    AllShIDList = pID2shID(to_ID,partType,SubhaloLenType,SnapOffsetsSubhalo)
    
    #####################
    #SAVE LISTS TO FILES#
    #####################
    
    np.save('/{0}/PartList_Snap{1}.npy'.format(subdir_name,snapshot_number),AllPartList)
    np.save('/{0}/ShIDList_Snap{1}.npy'.format(subdir_name,snapshot_number),AllShIDList)    

Creating for TNG100-2 snap 99
Processing snapshot 99 at:
/virgo/simulations/IllustrisTNG/TNG100-2/output/snapdir_099/snap_099.0.hdf5
 with offset file:
/virgo/simulations/IllustrisTNG/TNG100-2/postprocessing/offsets/offsets_099.hdf5
 and particle IDs file loc:
/Snapshots/99/PartType0/ParticleIDs


/raven/u/cwalker/virtual_environments/yt-env/lib/python3.7/site-packages/ipykernel_launcher.py:20: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
yt : [INFO     ] 2022-03-21 13:59:41,874 Calculating time from 1.000e+00 to be 4.356e+17 seconds
yt : [INFO     ] 2022-03-21 13:59:41,941 Parameters: current_time              = 4.355810528213311e+17 s
yt : [INFO     ] 2022-03-21 13:59:41,943 Parameters: domain_dimensions         = [1 1 1]
yt : [INFO     ] 2022-03-21 13:59:41,944 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2022-03-21 13:59:41,945 Parameters: domain_right_edge         = [75000. 75000. 75000.]
yt : [INFO     ] 2022-03-21 13:59:41,946 Parameters: cosmologic